In [80]:
import os
import csv
import json
from multiprocessing import Pool
from io import StringIO
from time import time

Fields in json object

Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name

In [81]:
basefolder = "/home/ubuntu/data/tmp/de/"
filename = "Womens Clothing E-Commerce Reviews.json"
filepath = os.path.join(basefolder, filename)

In [82]:
class ChunkedReader(object):
    
    def __init__(self, filepath, number_of_chunks):
        self.number_of_chunks = number_of_chunks
        self.filepath = filepath

    def read_block_lines(self, block):
        """
        
        """
        assert 0 <= block and block < self.number_of_chunks
        assert 0 < self.number_of_chunks

        with open(self.filepath,"r+") as filehandle:
            filehandle.seek(0,2)
            file_size = filehandle.tell()

            ini = int(file_size * block / self.number_of_chunks)
            end = int(file_size * (1 + block) / self.number_of_chunks)

            if ini <= 0:
                filehandle.seek(0)
            else:
                #ini can be in the middle of the line: read line first
                filehandle.seek(ini-1) 
                filehandle.readline()

            while filehandle.tell() < end:
                line = filehandle.readline()
                yield json.loads(line)

In [93]:
number_of_chunks = 4
number_of_processes = 4

def process_chunk(args):
    reader, block = args
    counter = 0
    age_sum = 0
    for js in reader.read_block_lines(block):
        try:
            age = int(js["Age"])
        except ValueError:
            print(js)
            continue
            
        counter += 1
        age_sum += age
        
    return age_sum, counter

start = time()
fin = ChunkedReader(filepath, number_of_chunks)
pool = Pool(number_of_processes)
age_super_sum = 0
super_count = 0
for age_sum, count in pool.imap_unordered(process_chunk,zip([fin]*number_of_chunks,range(number_of_chunks))):
    print(f"block: {age_sum} and {count}")
    super_count += count
    age_super_sum += age_sum
average_age = age_super_sum/super_count

print(f"average age {average_age}")
print("in {} seconds".format(time()-start))

block: 255287 and 5883
block: 250606 and 5831
block: 253511 and 5850
block: 255157 and 5922
average age 43.198543813335604
in 0.19129562377929688 seconds


In [94]:
age = 0
count = 0

start = time()
with open(filepath, "r") as fin:
    for line in fin:
        js = json.loads(line)
        try:
            age += int(js["Age"])
        except ValueError:
            continue
        count += 1
        
average_age = age/count
print(f"average age {average_age}")
print("in {} seconds".format(time()-start))

average age 43.198543813335604
in 0.20302867889404297 seconds
